In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from config import *

### Preparing the Dataset 

In [ ]:
from torchvision.datasets import VOCDetection
from utils.transform import train_transform
from utils.transform import val_transform

train_data = VOCDetection(root='./data', year='2012', image_set='train',
                             download=True, transforms=train_transform)

val_data = VOCDetection(root='./data', year='2012', image_set='train',
                             download=True, transforms=val_transform)

train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle = True,
        num_workers=0,
        pin_memory=True,
        drop_last=False,
    )

val_loader = torch.utils.data.DataLoader(
        val_data,
        batch_size=batch_size,
        shuffle = False,
        num_workers=0,
        pin_memory=True,
        drop_last=False,
    )

print('The ImageNet train set is ready. Size : {}'.format(len(train_loader)*batch_size))


dataloaders = dict()

dataloaders['train'] = train_loader
dataloaders['val'] = val_loader

dataset_sizes = {'train': len(train_loader)*batch_size,
                'val': len(val_loader)*batch_size}
print(dataset_sizes)

#### Visualizing the Dataset

In [3]:
from utils.utils import visualize_bb
from utils.dataset import inverse_target

samples  =[]

for i in range(15,22):
  img , targets = train_data[i]
  targets = [target.unsqueeze(0) for target in targets]
  bboxes, labels = inverse_target(targets)
  sample = {'image': img, 'bbox':bboxes[1], 'labels':labels[1]}     #should plot properly for all S[i]
  samples+=[sample]


visualize_bb(samples)

### Model Creation and Loss function Testing

In [ ]:
from model.loss import YoloV4_Loss
from model.yolov4 import YoloV4

model = YoloV4(num_classes = 20)
model = model.to(device)


#test loss on random inputs
g = torch.Generator().manual_seed(0)

preds  = [torch.randn((32, s,s,3,25),generator=g) for s in S]
ground_truths =[torch.randn((32, s,s,3,25),generator=g) for s in S]

for ground_truth in ground_truths:
  ground_truth[..., 0] = torch.empty_like(ground_truth[..., 0], dtype=torch.long).random_(2)
  ground_truth[...,5:] = torch.empty_like(ground_truth[..., 5:], dtype=torch.long).random_(2)

loss = YoloV4_Loss()
print('Loss1:', loss(preds, ground_truths))


#test loss on model 
img, targets = train_data[0]
img = img.to(device)
targets = [target.unsqueeze(dim=0).to(device) for target in targets]
pred = model(img.unsqueeze_(dim=0))
print('Loss2:' ,loss(pred, targets))


#Model output verification 
x = model(torch.rand(5, 3, 416, 416).to(device))
print("output shapes:", x[0].shape, x[1].shape, x[2].shape)


### Model Training

In [ ]:
print(f'Training \n')
torch.cuda.empty_cache()

model = YoloV4(num_classes = 20)
model = model.to(device)

#loss function 
criterion = YoloV4_Loss()

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.Adam(model.parameters(), lr=0.00001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=30, gamma=0.1)

In [ ]:
from model.train import train_model

model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, 
                    dataloaders, dataset_sizes, num_epochs=45)

### Visualizing outputs

In [ ]:
import numpy as np 
from utils.utils import visualize_outputs

indices = np.random.randint(0, 500, size=5)
visualize_outputs(indices, model, val_data)